In [1]:
import pandas as pd

In [2]:
from google.colab import files
data_to_load = files.upload()

Saving train.csv to train.csv


In [3]:
import io
train = pd.read_csv(io.StringIO(data_to_load["train.csv"].decode("utf-8")))
#train = pd.read_csv("train.csv")
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = train.copy()

In [5]:
#Drop Nan Values
df = df.dropna()

In [7]:
## Get the independent feature 
X = df.drop('label' , axis=1)

In [8]:
## Get the Dependent Feature 
Y = df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
Y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.2.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense  # Since this is a classifcation problem , the final layer should have one node which gives a values of whether it is >0.5(Fake) or less than 0.5(Not Fake)


In [14]:
## Vocabulary Size 
voc_size = 5000 #when increasing the voc_size to 10K, there was only a minute difference in accuracy 

# ***OneHot Representation***

In [15]:
messages = X.copy()
messages.reset_index(inplace = True)

In [16]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer  # Stemming
ps = PorterStemmer()
corpus = []
for i in range(0 , len(messages)):
  review = re.sub('[^a-zA-Z]' , ' ' , messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]  #Forms a list
  review = ' '.join(review)
  corpus.append(review)



In [38]:
corpus 

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [39]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1935, 673, 1043, 3241, 1819, 1934, 2407, 271, 3391, 2317],
 [3199, 2006, 530, 4759, 4255, 1368, 2396],
 [390, 1909, 4607, 2511],
 [1479, 80, 3066, 528, 3972, 1495],
 [3811, 4255, 2034, 4799, 1045, 2286, 4255, 4749, 2225, 552],
 [1288,
  3598,
  4268,
  387,
  4928,
  1663,
  783,
  1255,
  3484,
  38,
  1768,
  3329,
  3273,
  4022,
  2396],
 [2148, 3026, 7, 1312, 2131, 2403, 3678, 998, 3502, 1978, 4619],
 [661, 3145, 3546, 4332, 1838, 1823, 1663, 1572, 3502, 1978, 4619],
 [3188, 2653, 932, 4082, 2906, 126, 2663, 3200, 1663, 2683],
 [3243, 1961, 3688, 308, 3560, 2810, 2804, 3497],
 [4718, 2102, 2590, 2806, 3384, 537, 1014, 255, 4659, 4953, 1912],
 [528, 3077, 1819, 126, 1663, 1838],
 [4593, 384, 2521, 1189, 4725, 2852, 1097, 2702, 1887],
 [2231, 497, 4382, 4863, 3881, 4192, 2695, 3502, 1978, 4619],
 [3467, 1252, 3438, 2164, 2747, 3502, 1978, 4619],
 [3063, 3409, 4510, 3363, 4748, 4850, 255, 4277, 3478, 2645],
 [4274, 4525, 2006],
 [37, 1961, 4812, 3833, 1663, 201, 811, 2396],
 [1909,

## ****Embedding Representation****

In [42]:
sent_leng = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre',maxlen = sent_leng)
print(embedded_docs)

[[   0    0    0 ...  271 3391 2317]
 [   0    0    0 ... 4255 1368 2396]
 [   0    0    0 ... 1909 4607 2511]
 ...
 [   0    0    0 ... 3502 1978 4619]
 [   0    0    0 ... 4617 2154 3987]
 [   0    0    0 ... 4793  872 4932]]


In [43]:
len(embedded_docs)

18285

In [44]:
## Creating Model 
embedding_vector_features = 40    #model handles this part automatically
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features ,input_length = sent_leng))
model.add(LSTM(100))  #1 LSTM Layer with 100 neurons
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
len(embedded_docs) , Y.shape

(18285, (18285,))

In [49]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [51]:
X_final.shape , Y_final.shape

((18285, 20), (18285,))

In [52]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train ,Y_test = train_test_split(X_final , Y_final , test_size = 0.33 , random_state = 42)

# ***Model Training***

In [54]:
# Training
model.fit(X_train , Y_train , validation_data=(X_test,Y_test) , epochs= 10 , batch_size = 64)  

Epoch 1/10
192/192 [==============================] - 7s 38ms/step - loss: 0.3339 - accuracy: 0.8402 - val_loss: 0.2037 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1387 - accuracy: 0.9478 - val_loss: 0.1946 - val_accuracy: 0.9249
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0961 - accuracy: 0.9638 - val_loss: 0.2309 - val_accuracy: 0.9221
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0659 - accuracy: 0.9776 - val_loss: 0.2728 - val_accuracy: 0.9201
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0417 - accuracy: 0.9863 - val_loss: 0.2930 - val_accuracy: 0.9128
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0235 - accuracy: 0.9930 - val_loss: 0.3311 - val_accuracy: 0.9160
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0143 - accuracy: 0.9956 - val_loss: 0.4232 - val_accuracy: 0.9125

#Adding DropOut Layer  - A method for HyperParameter Tuning 

In [58]:
from tensorflow.keras.layers import Dropout

## Creating Model 
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features , input_length = sent_leng))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1 , activation='sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics=['accuracy'])



In [59]:
# Training
model.fit(X_train , Y_train , validation_data=(X_test,Y_test) , epochs= 10 , batch_size = 64)  

Epoch 1/10
192/192 [==============================] - 8s 40ms/step - loss: 0.3449 - accuracy: 0.8459 - val_loss: 0.1929 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1462 - accuracy: 0.9431 - val_loss: 0.1896 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1036 - accuracy: 0.9628 - val_loss: 0.2112 - val_accuracy: 0.9231
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0769 - accuracy: 0.9738 - val_loss: 0.2158 - val_accuracy: 0.9200
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0569 - accuracy: 0.9808 - val_loss: 0.3086 - val_accuracy: 0.9210
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0369 - accuracy: 0.9873 - val_loss: 0.2947 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0327 - accuracy: 0.9887 - val_loss: 0.3273 - val_accuracy: 0.9137

## **Performance Metrics And Accuracy**

In [65]:
Y_pred = model.predict_classes(X_test)

In [66]:
from sklearn.metrics import confusion_matrix

In [67]:
confusion_matrix(Y_test , Y_pred )

array([[3076,  343],
       [ 177, 2439]])

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test , Y_pred)

0.9138359569179785

## **Bidirectional RNN**

In [76]:
#Importing Library
from tensorflow.keras.layers import Bidirectional

## Creating Model 
embedding_vector_features = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vector_features , input_length = sent_leng))
model2.add(Dropout(0.3))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dropout(0.3))
model2.add(Dense(1 , activation='sigmoid'))
model2.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics=['accuracy'])


In [77]:
# Training
model2.fit(X_train , Y_train , validation_data=(X_test,Y_test) , epochs= 10 , batch_size = 64)  

Epoch 1/10
192/192 [==============================] - 11s 60ms/step - loss: 0.3196 - accuracy: 0.8470 - val_loss: 0.1962 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 10s 54ms/step - loss: 0.1444 - accuracy: 0.9455 - val_loss: 0.1995 - val_accuracy: 0.9171
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1028 - accuracy: 0.9615 - val_loss: 0.2416 - val_accuracy: 0.9158
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0793 - accuracy: 0.9703 - val_loss: 0.2768 - val_accuracy: 0.9054
Epoch 5/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0604 - accuracy: 0.9786 - val_loss: 0.2803 - val_accuracy: 0.9107
Epoch 6/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0468 - accuracy: 0.9842 - val_loss: 0.3674 - val_accuracy: 0.9052
Epoch 7/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0397 - accuracy: 0.9863 - val_loss: 0.3169 - val_accuracy:

In [81]:
Y_pred2 = model2.predict_classes(X_test)

In [83]:
confusion_matrix(Y_test , Y_pred2 )

array([[3108,  311],
       [ 246, 2370]])

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test , Y_pred2)

0.907705053852527

In [85]:
from sklearn.metrics import classification_report
print(classification_report(Y_test , Y_pred2))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



Dropout Layer Follows well when new testdata is encountered.